# Prepare the selected metapaths for feature computation for all compound-disease pairs

In [1]:
import json
import itertools
import bz2
import configparser

import pandas

## Read node info

In [2]:
# Read compound and disease degrees
compound_df = pandas.read_table('../summary/compounds.tsv')
disease_df = pandas.read_table('../summary/diseases.tsv')
len(compound_df), len(disease_df)

(1538, 136)

In [3]:
config = configparser.ConfigParser()
config.read('../config.ini')
commit = config['hetnet']['pharmacotherapydb_commit']
url = 'https://github.com/dhimmel/indications/blob/{}/catalog/indications.tsv?raw=true'.format(commit)
indication_df = pandas.read_table(url)
indication_df = indication_df.rename(columns={'doid_id': 'disease_id', 'drugbank_id': 'compound_id'})[['compound_id', 'disease_id', 'category']]
indication_df.head(2)

,compound_id,disease_id,category
0,DB00843,DOID:10652,DM
1,DB00674,DOID:10652,DM


In [4]:
rows = list()
for c, d in itertools.product(compound_df.itertuples(), disease_df.itertuples()):
    row = c.compound_id, c.compound_name, d.disease_id, d.disease_name
    rows.append(row)
pair_df = pandas.DataFrame(rows, columns=['compound_id', 'compound_name', 'disease_id', 'disease_name'])
pair_df = pair_df.merge(indication_df, how='left')
pair_df['status'] = (pair_df.category == 'DM').astype(int)
pair_df.head(2)

,compound_id,compound_name,disease_id,disease_name,category,status
0,DB01048,Abacavir,DOID:10652,Alzheimer's disease,NaN,0
1,DB01048,Abacavir,DOID:9206,Barrett's esophagus,NaN,0


In [5]:
pair_df['status'].value_counts()

0    208413
1       755
Name: status, dtype: int64

In [6]:
len(pair_df)

209168

In [7]:
with bz2.open('features/compound-disease-pairs.tsv.bz2', 'wt') as write_file:
    pair_df.to_csv(write_file, sep='\t', index=False)

## Select metapaths

In [8]:
path = '../all-features/selection/sweep-features.tsv'
feature_df = pandas.read_table(path)
feature_df = feature_df.query("perm_affected >= 0.5")
feature_df.to_csv('features/selected-features.tsv', sep='\t', index=False, float_format='%.3g')
metapath_df = feature_df.query("feature_type == 'dwpc'")
#feature_df = feature_df.rename(columns={'feature': 'metapath'})
len(feature_df), len(metapath_df)

(75, 66)

In [9]:
metapaths = set(metapath_df.feature)
metapath_df.head(2)

,feature,all_features,perm_affected,feature_type,nonzero,auroc,auroc_permuted,delta_auroc,pval_auroc,fdr_pval_auroc
0,CbGaD,1.0,1.0,dwpc,0.31444,0.71627,0.57794,0.13832,2.940400e-06,0.000255
3,CbGbCtD,1.0,1.0,dwpc,0.72980,0.86666,0.66722,0.19945,2.998800e-07,0.000073


## Metaedges in chosen metapaths

In [10]:
m2m_df = pandas.read_table('../all-features/data/metaedge-in-metapath.tsv')
m2m_df = m2m_df.query("metapath in @metapaths")

In [11]:
# Counting multiple occurrences in the same metapath
m2m_df.metaedge.value_counts().reset_index()

,index,metaedge
0,Compound - treats - Disease,44
1,Compound - binds - Gene,25
2,Compound - palliates - Disease,23
3,Disease - resembles - Disease,19
4,Disease - associates - Gene,19
5,Compound - resembles - Compound,15
6,Disease - upregulates - Gene,13
7,Compound - downregulates - Gene,12
8,Pharmacologic Class - includes - Compound,12
9,Disease - localizes - Anatomy,11


In [12]:
# Counting only one metaedge occurrence per metapath
m2m_df.drop_duplicates().metaedge.value_counts().reset_index()

,index,metaedge
0,Compound - treats - Disease,44
1,Compound - binds - Gene,25
2,Compound - palliates - Disease,23
3,Disease - resembles - Disease,19
4,Disease - associates - Gene,19
5,Compound - resembles - Compound,15
6,Disease - upregulates - Gene,13
7,Compound - downregulates - Gene,12
8,Pharmacologic Class - includes - Compound,12
9,Disease - localizes - Anatomy,11


In [13]:
# Number of included metaedges
m2m_df.metaedge.nunique()

20

In [14]:
with open('../all-features/data/metapaths.json') as read_file:
    metapath_obj = json.load(read_file)
metapath_obj = [x for x in metapath_obj if x['abbreviation'] in metapaths]
with open('features/metapaths.json', 'wt') as write_file:
    metapath_obj = json.dump(metapath_obj, write_file, indent=2, sort_keys=True)